In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, GlobalAveragePooling1D, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

# Load the preprocessed data
X_train = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_train.txt")
y_train = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_train.txt")
X_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_test.txt")
y_test = np.loadtxt("../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_test.txt")

# Reshape the input data for Conv1D layers
n_timestep = 100
n_features = 9
X_train = X_train.reshape(X_train.shape[0], n_timestep, n_features)
X_test = X_test.reshape(X_test.shape[0], n_timestep, n_features)

# One-hot encode the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)


def create_lstm_model(n_timesteps, n_features, n_outputs):
    model = Sequential([
        # LSTM layer with return_sequences=False to reduce complexity
        LSTM(64, activation="tanh", input_shape=(n_timesteps, n_features)),

        # Dense layers for classification
        Dense(128, activation="relu"),
        Dense(64, activation="relu"),
        Dense(n_outputs, activation="softmax")  # For multi-class classification
    ])
    return model

n_outputs = y_train.shape[1]
conv_mlp = create_lstm_model(n_timestep, n_features, n_outputs)
conv_mlp.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = conv_mlp.fit(X_train, y_train, epochs=50, batch_size=512, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = conv_mlp.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, digits=5))

2024-09-27 09:58:50.656057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 09:58:50.666910: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 09:58:50.670306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 09:58:50.678884: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 09:58:51.186676: W tensorflow/compiler/tf2

Epoch 1/50


2024-09-27 09:59:13.939769: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


156/156 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.2711 - loss: 2.3821 - val_accuracy: 0.4392 - val_loss: 1.7604
Epoch 2/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5772 - loss: 1.2755 - val_accuracy: 0.5583 - val_loss: 1.4986
Epoch 3/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6700 - loss: 0.9978 - val_accuracy: 0.6000 - val_loss: 1.3339
Epoch 4/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7313 - loss: 0.8230 - val_accuracy: 0.6182 - val_loss: 1.3622
Epoch 5/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7601 - loss: 0.7254 - val_accuracy: 0.6409 - val_loss: 1.3411
Epoch 6/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7889 - loss: 0.6445 - val_accuracy: 0.6377 - val_loss: 1.3399
Epoch 7/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7981 - loss: 0.6087 - val_accuracy: 0.6312 - val_loss: 1.3789
Epoch 8/50
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8091 - loss: 0.5772 - val_accuracy: 0.6430 - val_

In [2]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(conv_mlp)
tflite_model = converter.convert()

# Save the TFLite model
with open("SubjectDependent/GeneratedTFLiteFilesAndOGModels/lstm_mlp_model_deploy.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model has been saved as 'lstm_mlp_model_deploy.tflite'")

# Optional: Save the Keras model
conv_mlp.save("GeneratedTFLiteFilesAndOGModels/lstm_mlp_model_deploy.h5")
print("Keras model has been saved as 'lstm_mlp_model_deploy.h5'")

# If you still need to generate a C header file
try:
    from everywhereml.code_generators.tensorflow import convert_model

    c_header = convert_model(conv_mlp, X_test, y_test, model_name='lstm_mlp_model_deploy')

    with open("SubjectDependent/GeneratedHeaderFiles/lstm_mlp_model.h", "w") as file:
        file.write(c_header)

    print("C header file has been saved as GeneratedHeaderFiles/lstm_mlp_model.h'")
except ImportError:
    print("everywhereml library not found. Skipping C header file generation.")


INFO:tensorflow:Assets written to: /tmp/tmpxhdje4lq/assets


INFO:tensorflow:Assets written to: /tmp/tmpxhdje4lq/assets


Saved artifact at '/tmp/tmpxhdje4lq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  129068357453072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129068357453952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129068357452368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129068357456768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129068357458176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129068357455008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129068357459760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129068357456416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129068357461344: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727409774.906458  116757 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727409774.906470  116757 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 10:02:54.906653: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpxhdje4lq
2024-09-27 10:02:54.907035: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 10:02:54.907042: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpxhdje4lq
2024-09-27 10:02:54.910585: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-09-27 10:02:54.911157: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 10:02:54.930032: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpxhdje4lq
2024-09-27 10:02:54.936939: I tensorflow/cc/saved_model/loader.cc

ConverterError: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_1/lstm_1/CudnnRNNV3@__inference_function_44032"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_44079"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): 'tf.CudnnRNNV3' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["CudnnRNNV3:", "sequential_1/lstm_1/CudnnRNNV3@__inference_function_44032"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_44079"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): Error code: ERROR_NEEDS_CUSTOM_OPS
<unknown>:0: error: failed while converting: 'main': 
Some ops in the model are custom ops, See instructions to implement custom ops: https://www.tensorflow.org/lite/g